In [1]:
import os
import numpy as np
import skimage.io as io
import skimage.transform as trans
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

ModuleNotFoundError: No module named 'skimage'

In [ ]:
IMAGE_PATH = "./timages/"
MASK_PATH = "./masks/"
READ_SIZE = (512, 512)
INPUT_SIZE = (512, 512, 3)
BATCH_SIZE = 32
VAL_SPLIT = 0.2

In [ ]:
#Funcs for metrics, preprocessing
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def IoU_coeff(y_true, y_pred):
    axes = (1,2) 
    intersection = tf.keras.backend.sum(tf.math.abs(y_pred * y_true), axis=axes) 
    mask = tf.keras.backend.sum(tf.math.abs(y_true), axis=axes) + tf.keras.backend.sum(tf.math.abs(y_pred), axis=axes)
    union = mask - intersection
    smooth = .001
    iou = (intersection + smooth) / (union + smooth)
    return iou

def process_mask(img):
    mask = img.astype(np.float32)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    return mask

In [ ]:
#Create Datagenerator
image_data = ImageDataGenerator(validation_split=VAL_SPLIT)
mask_data = ImageDataGenerator(preprocessing_function=process_mask, validation_split=VAL_SPLIT)

image_gen = image_data.flow_from_directory(IMAGE_PATH, target_size=READ_SIZE, class_mode=None, 
                                           batch_size=BATCH_SIZE, shuffle=False, subset='training')
val_image_gen = image_data.flow_from_directory(IMAGE_PATH, target_size=READ_SIZE, class_mode=None, 
                                           batch_size=BATCH_SIZE, shuffle=False, subset='validation')
mask_gen = image_data.flow_from_directory(MASK_PATH, target_size=READ_SIZE, class_mode=None, 
                                          batch_size=BATCH_SIZE, shuffle=False, subset='training')
val_mask_gen = image_data.flow_from_directory(MASK_PATH, target_size=READ_SIZE, class_mode=None, 
                                          batch_size=BATCH_SIZE, shuffle=False, subset='validation')

train_generator = zip(image_gen, mask_gen)
val_generator = zip(val_image_gen, val_mask_gen)

In [ ]:
#Create Unet Model
inputs = Input(INPUT_SIZE)
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
merge6 = concatenate([drop4,up6], axis = 3)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = concatenate([conv3,up7], axis = 3)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 = concatenate([conv2,up8], axis = 3)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = concatenate([conv1,up9], axis = 3)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

model = Model(inputs, conv10)

In [ ]:
model.compile(optimizer=Adam(1e-4, decay=1e-6), loss=dice_coef_loss, metrics=[dice_coef, IoU_coeff])

In [ ]:
history = model.fit(train_generator, steps_per_epoch = image_gen.samples // BATCH_SIZE, 
                    validation_data = val_generator, validation_steps = val_image_gen.samples // BATCH_SIZE, 
                    epochs=10)

In [ ]:
#Save Model and training History
model_json = model.to_json()
with open("Unet_model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("Unet_model.h5")

with open('/Unet_trainHistory', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss')
plt.ylabel('val_loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('dice_coef')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
                         
plt.plot(history.history['IoU_coeff'])
plt.plot(history.history['val_IoU_coeff'])
plt.title('iou')
plt.ylabel('iou')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()